In [1]:
import pandas as pd
import numpy as np
import joblib

In [10]:
with open('data/my_val.pkl', 'rb') as f:
    data = joblib.load(f)
locals().update(data)
del data
np.random.seed(123)

test = pd.read_csv('data/test_ver02.csv', index_col='STU_ID')
test = test.drop(columns='F3ONET2CURR')

In [11]:
# Scaler?
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
x_tr = minmax.fit_transform(x)
val_x_tr = minmax.transform(val_x)
test_tr = minmax.transform(test)

In [22]:
idx2class = {}
for i, v in enumerate(y.unique()):
    idx2class[i] = v
class2idx = {v:i for i, v in idx2class.items()}
y_tr = y.replace(class2idx)
val_y_tr = val_y.replace(class2idx)

In [23]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense

In [24]:
my_nn = Sequential([
            Input(shape=(x_tr.shape[1],)),
            Dense(128, activation='relu'),
            Dense(64, activation='relu'),
            Dense(32, activation='relu'),
            Dense(y_tr.nunique(), activation='softmax')
    ])
my_nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               19456     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 20)                660       
Total params: 30,452
Trainable params: 30,452
Non-trainable params: 0
_________________________________________________________________


In [27]:
epochs = 100
batch = 128
my_nn.compile(loss='sparse_categorical_crossentropy',optimizer='RMSProp',
              metrics=['acc'])
my_nn.fit(x_tr, y_tr, batch_size=batch, epochs=epochs)

Epoch 1/100
59/59 [==============================] - 0s 625us/step - loss: 2.7810 - acc: 0.1562
Epoch 2/100
59/59 [==============================] - 0s 609us/step - loss: 2.6065 - acc: 0.1863
Epoch 3/100
59/59 [==============================] - 0s 609us/step - loss: 2.5265 - acc: 0.2065
Epoch 4/100
59/59 [==============================] - 0s 609us/step - loss: 2.4738 - acc: 0.2174
Epoch 5/100
59/59 [==============================] - 0s 609us/step - loss: 2.4401 - acc: 0.2313
Epoch 6/100
59/59 [==============================] - 0s 592us/step - loss: 2.4161 - acc: 0.2336
Epoch 7/100
59/59 [==============================] - 0s 592us/step - loss: 2.3947 - acc: 0.2431
Epoch 8/100
59/59 [==============================] - 0s 584us/step - loss: 2.3790 - acc: 0.2473
Epoch 9/100
59/59 [==============================] - 0s 575us/step - loss: 2.3632 - acc: 0.2510
Epoch 10/100
59/59 [==============================] - 0s 609us/step - loss: 2.3483 - acc: 0.2556
Epoch 11/100
59/59 [===================

In [28]:
my_nn.evaluate(val_x_tr, val_y_tr)

78/78 [==============================] - 0s 345us/step - loss: 3.8940 - acc: 0.1760


[3.8939826488494873, 0.1759742796421051]

In [37]:
pred = my_nn.predict(test_tr)
pred_class = [idx2class[i] for i in pred.argmax(axis=1)]

In [40]:
submission = pd.DataFrame(pred_class, index=test.index, columns=['PRED']).reset_index()
submission.to_csv('my_pred02.csv', index=False)

- AutoEncoder 사용 : 과소완전AutoEncoder

In [44]:
num_dim = 64
autoE = Sequential([
            Input(shape=(x_tr.shape[1],)),
            Dense(num_dim, activation='relu'),
            Dense(x_tr.shape[1], activation='relu')])
autoE.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 64)                9728      
_________________________________________________________________
dense_6 (Dense)              (None, 151)               9815      
Total params: 19,543
Trainable params: 19,543
Non-trainable params: 0
_________________________________________________________________


In [46]:
epochs = 100
batch = 128
autoE.compile(loss='mean_squared_error',optimizer='RMSProp',
              metrics=['mse'])
autoE.fit(x_tr, x_tr, batch_size=128, epochs=epochs)

Epoch 1/100
59/59 [==============================] - 0s 654us/step - loss: 0.0944 - mse: 0.0944
Epoch 2/100
59/59 [==============================] - 0s 540us/step - loss: 0.0633 - mse: 0.0633
Epoch 3/100
59/59 [==============================] - 0s 552us/step - loss: 0.0523 - mse: 0.0523
Epoch 4/100
59/59 [==============================] - 0s 567us/step - loss: 0.0460 - mse: 0.0460
Epoch 5/100
59/59 [==============================] - 0s 566us/step - loss: 0.0421 - mse: 0.0421
Epoch 6/100
59/59 [==============================] - 0s 558us/step - loss: 0.0394 - mse: 0.0394
Epoch 7/100
59/59 [==============================] - 0s 558us/step - loss: 0.0374 - mse: 0.0374
Epoch 8/100
59/59 [==============================] - 0s 553us/step - loss: 0.0320 - mse: 0.0320
Epoch 9/100
59/59 [==============================] - 0s 541us/step - loss: 0.0304 - mse: 0.0304
Epoch 10/100
59/59 [==============================] - 0s 578us/step - loss: 0.0291 - mse: 0.0291
Epoch 11/100
59/59 [===================

In [51]:
active_model = Model(inputs=autoE.input, outputs=autoE.layers[0].output)
x_auto = active_model.predict(x_tr)
val_x_auto = active_model.predict(val_x_tr)
test_auto = active_model.predict(test_tr)

In [54]:
my_nn02 = Sequential([
            Input(shape=(x_auto.shape[1],)),
            Dense(48, activation='relu'),
            Dense(32, activation='relu'),
            Dense(y_tr.nunique(), activation='softmax')
    ])
my_nn02.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 48)                3120      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                1568      
_________________________________________________________________
dense_9 (Dense)              (None, 20)                660       
Total params: 5,348
Trainable params: 5,348
Non-trainable params: 0
_________________________________________________________________


In [55]:
epochs = 100
batch = 128
my_nn02.compile(loss='sparse_categorical_crossentropy',optimizer='RMSProp',
              metrics=['acc'])
my_nn02.fit(x_auto, y_tr, batch_size=batch, epochs=epochs)

Epoch 1/100
59/59 [==============================] - 0s 473us/step - loss: 2.8347 - acc: 0.1305
Epoch 2/100
59/59 [==============================] - 0s 437us/step - loss: 2.7168 - acc: 0.1507
Epoch 3/100
59/59 [==============================] - 0s 452us/step - loss: 2.6543 - acc: 0.1692
Epoch 4/100
59/59 [==============================] - 0s 429us/step - loss: 2.6111 - acc: 0.1768
Epoch 5/100
59/59 [==============================] - 0s 448us/step - loss: 2.5782 - acc: 0.1882
Epoch 6/100
59/59 [==============================] - 0s 473us/step - loss: 2.5514 - acc: 0.1895
Epoch 7/100
59/59 [==============================] - 0s 439us/step - loss: 2.5298 - acc: 0.1960
Epoch 8/100
59/59 [==============================] - 0s 440us/step - loss: 2.5125 - acc: 0.1984
Epoch 9/100
59/59 [==============================] - 0s 431us/step - loss: 2.5010 - acc: 0.2053
Epoch 10/100
59/59 [==============================] - 0s 440us/step - loss: 2.4893 - acc: 0.2079
Epoch 11/100
59/59 [===================

In [56]:
my_nn02.evaluate(val_x_auto, val_y_tr)

78/78 [==============================] - 0s 345us/step - loss: 2.5374 - acc: 0.2001


[2.5374293327331543, 0.20008035004138947]

In [58]:
pred = my_nn02.predict(test_auto)
pred_class = [idx2class[i] for i in pred.argmax(axis=1)]

In [60]:
submission = pd.DataFrame(pred_class, index=test.index, columns=['PRED']).reset_index()
submission.to_csv('my_pred03.csv', index=False)